In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.contrib.layers as layers
from sklearn.metrics import roc_auc_score

import generate
import plot
import models
import environment

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



# Make the graphs and run the computation

In [ ]:
COUNTER = 0

In [ ]:
NOISE_DIM = 3
BATCH_SIZE = 512
TEST_SIZE = 100000
N_PRE_STEPS = 100
N_STEPS = 500
N_DRAW = 5
COUNTER+=1 # so that new graphs are created each time
generate_data = generate.triangular1D
#generate_data = generate.uniform1D
#generate_data = generate.beta1D

G = models.GeneratorFullyConnected('Generator_{}'.format(COUNTER), depth=3, n_units=20, output_dim=1)
A = models.AdversaryFullyConnected('Adversary_{}'.format(COUNTER), depth=2, n_units=20, n_classes=2)
env = environment.TFEnvironment(G, A, generate_data, generate.input_noise(NOISE_DIM))
env.build()
env.initialise_variables()

aurocs = []
adv_losses = []
for i in range(N_PRE_STEPS):
    
    # pretraining step
    env.pretrain_step(BATCH_SIZE)
    
    # get analytics
    to_run = [env.generator.output, env.adversary.labels, env.adversary.logits, env.adversary.loss]
    fake_data, labels, logits, adv_loss = env.run(BATCH_SIZE, *to_run)
    
    # plot only N times
    aurocs.append(roc_auc_score(labels, logits))
    adv_losses.append(adv_loss)
        
print('Done with pretraining')

# now run the training steps, and plot the fake and real data
for i in range(N_STEPS):

    # run the computation (discriminator, generator, performance monitoring)
    for _ in range(20):
        env.train_adversary(BATCH_SIZE)
    env.train_generator(BATCH_SIZE)
    
    # get analytics
    to_run = [env.adversary.labels,
              env.adversary.logits,
              env.adversary.loss]   
    labels, logits, adv_loss = env.run(BATCH_SIZE, *to_run)
    
    # plot only ten times
    aurocs.append(roc_auc_score(labels, logits))
    adv_losses.append(adv_loss)
    do_plot = (i in [n* int(N_STEPS/N_DRAW) for n in range(N_DRAW)]) or (i == N_STEPS-1)

    if do_plot:
        print('Training step {}/{}'.format(i, N_STEPS))
        [fake_data] = env.run(TEST_SIZE, env.generator.output)
        fig, ax = plt.subplots(1, 2, figsize=(10, 5))
        plot.data1D(ax[0], generate_data(TEST_SIZE), fake_data.ravel())
        plot.roc_curve(ax[1], labels[:,1], logits[:,1])
        plt.show()
    
print('Done with training')

In [ ]:
nplots=2
fig, ax = plt.subplots(1, nplots, sharex=True, figsize=(nplots*6, 5))
ax[0].plot(range(len(aurocs)), aurocs, c='r')
ax[0].set_xlabel('Training steps')
ax[0].set_ylabel('AUROC')
ax[1].plot(range(len(adv_losses)), adv_losses, c='g')
ax[1].set_xlabel('Training steps')
ax[1].set_ylabel('Adversary loss')
plt.show()

In [ ]:
LARGE = 1000000
fake_data = env.run(LARGE, env.generator.output)[0]
real_data = generate_data(LARGE)

fig, ax = plt.subplots(2, 1, figsize=(5, 5), gridspec_kw={'height_ratios':[3,1]})
plot.data1D(ax[0], real_data.ravel(), fake_data.ravel())
plot.ratio1D(ax[1], real_data.ravel(), fake_data.ravel())
plt.savefig('Test_{}.png'.format(generate_data.__name__))
plt.show()

